In [1]:
import torch

# Local imports
from train import train_model

Using device: cpu


In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cuda:0


In [3]:
args = {
    "use_small_dataset": False,
    "batch_size": 64,
    "stoppage_epochs": 42,
    "max_epochs": 512,
    "seed": 42,
    "data_path": "../data",
    "frac_train": 0.8,
    "frac_validation": 0.1,
    "frac_test": 0.1,
    "huber_beta": 2.0,
    "weight_decay": 1e-3,
    "lr": 3e-4,
    "scheduler_patience": 10,
    "scheduler_factor": 0.5,
    "hidden_size": 96,
    "num_layers": 7,
    "num_attn_heads": 6,
    "dropout": 0.2,
    "leaky_relu_slope": 0.2,
    "pooling_dim": 96,
}

In [4]:
import argparse

training_args = argparse.Namespace(**args)
train_model(training_args, device)

Model parameters: 182714


Epoch 1/512:   0%|                                                                                | 0/910 [00:00<?, ?it/s]


NameError: name 'y_preds' is not defined